In [0]:
%sql
select count(*) from dev.demo_db.fire_calls_tbl

In [0]:
%fs ls /Volumes/dev/demo_db/landing

In [0]:
%sql
create table if not exists dev.demo_db.invoices_raw(
  InvoiceNo string,
  StockCode string,
  Description string,
  Quantity int,
  InvoiceDate timestamp,
  UnitPrice double,
  CustomerID string
)

### Ingest using Copy

In [0]:
%sql
COPY INTO dev.demo_db.invoices_raw
FROM
(select InvoiceNo::string,
  StockCode::string,
  Description::string,
  Quantity::int,
  to_timestamp(InvoiceDate,"d-M-y H.m")InvoiceDate,
  UnitPrice::double,
  CustomerID::string
from "/Volumes/dev/demo_db/landing")
FILEFORMAT = CSV
FORMAT_OPTIONS("header" = "true")

In [0]:
%sql
select * from dev.demo_db.invoices_raw

### COPY INTO is idempotent

In [0]:
%sql
COPY INTO dev.demo_db.invoices_raw
FROM
(select InvoiceNo::string,
  StockCode::string,
  Description::string,
  Quantity::int,
  to_timestamp(InvoiceDate,"d-M-y H.m")InvoiceDate,
  UnitPrice::double,
  CustomerID::string
from "/Volumes/dev/demo_db/landing")
FILEFORMAT = CSV
FORMAT_OPTIONS("header" = "true")

In [0]:
%sql
select * from dev.demo_db.invoices_raw

In [0]:
%fs ls /Volumes/dev/demo_db/landing/invoices

In [0]:
%fs cp /Volumes/dev/demo_db/landing/invoices/invoices_2021.csv /Volumes/dev/demo_db/landing

In [0]:
%fs ls /Volumes/dev/demo_db/landing

#### Ingestion Code will not break but silently ignore the additional column

##### Alter the table manually to accomodate additional field

In [0]:
%sql
alter table dev.demo_db.invoices_raw add columns (Country string)

In [0]:
%sql
COPY INTO dev.demo_db.invoices_raw
FROM
(select InvoiceNo::string,
  StockCode::string,
  Description::string,
  Quantity::int,
  to_timestamp(InvoiceDate,"d-M-y H.m")InvoiceDate,
  UnitPrice::double,
  CustomerID::string, Country::string
from "/Volumes/dev/demo_db/landing")
FILEFORMAT = CSV
FORMAT_OPTIONS("header" = "true",'mergeSchema'='true')

In [0]:
%sql
select * from dev.demo_db.invoices_raw